pip install -r requirements.txt

In [ ]:
import sys
import os
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium import webdriver
from collections import namedtuple
from time import sleep
import pickle
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementNotInteractableException

# INPUTS

In [ ]:
my_msg = '<name> in less than two days this Wednesday our event at the heart of Athens. Remind me to bring a power strip. Will you ? :)'
my_msg

In [ ]:
# members_url = 'https://www.meetup.com/social-cinema/members/'
members_url = 'https://www.meetup.com/coworking-in-athens/members/'
# members_url = 'https://www.meetup.com/athens-travelers/members/'

shifted = 0 # useful to start from some other index, when you have for example 200 members and looping through them crashed at 110, then you need to set it to 110 in order to loop through the rest 90 of them
sleep_between_keystrokes_secs=0.09
sleep_secs_between_messages=10

# exclude_members = ['some name', 'some other name']
exclude_members = []

# Body

In [ ]:
Member = namedtuple('Member', 'name link')

In [ ]:
# driver = webdriver.Chrome(ChromeDriverManager().install()) # selenium 3
# driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install())) # selenium 4
driver = webdriver.Chrome(service=ChromeService( '/Users/gp/chrome_drivers/chromedriver' ))

In [ ]:
driver.get("https://www.meetup.com")

In [ ]:
## FIRST TIME ONLY
## Login with Facebook to the window that is opened
# pickle.dump( driver.get_cookies() , open("cookies.pkl","wb"))

In [ ]:
cookies = pickle.load(open("cookies.pkl", "rb"))
for ii, cookie in enumerate(cookies):
    # print(ii)
    try:
        driver.add_cookie(cookie)
    except:
        print(cookie)

In [ ]:
### get rid of accept button
# driver.find_element(by=By.CSS_SELECTOR, value='button#onetrust-accept-btn-handler').click() # implicit wait was not enough
WebDriverWait(driver, 15).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'button#onetrust-accept-btn-handler'))).click();

In [ ]:
driver.get(members_url)

In [ ]:
######## START LOOP ###########

In [ ]:
while True:
    members = driver.find_elements(by=By.CSS_SELECTOR, value='.member-item')
    count_before = len(members)
    print("Count before", count_before)
    
    members[-1].location_once_scrolled_into_view #scroll to last element
    
    buttons = driver.find_elements(by=By.CSS_SELECTOR, value='.infiniteScrollLoadMoreButton')
    count_buttons = len(buttons)
    print("Count buttons", count_buttons)
    
    if len(buttons) > 0:
        buttons[0].click()
        sleep(5)
    else:
        sleep(5)
        
        buttons = driver.find_elements(by=By.CSS_SELECTOR, value='.infiniteScrollLoadMoreButton')
        count_buttons = len(buttons)
        print("Count buttons a second time", count_buttons)
        
        if len(buttons) > 0:
            buttons[0].click()
            sleep(5)
        else:
            pass #do nothing you have already slept
    
    members = driver.find_elements(by=By.CSS_SELECTOR, value='.member-item')
    count_after = len(members)
    print("Count after", count_after)
    if (count_before == count_after) and len(buttons) == 0:
        print("STOP")
        break
    else:
        print("one more time")

In [ ]:
##### END LOOP ####

In [ ]:
membersElems = driver.find_elements(by=By.CSS_SELECTOR, value='.member-item')
len(membersElems)

In [ ]:
def extract_name_and_link(member):
    names = member.find_elements(by=By.CSS_SELECTOR, value='h4.text--bold')
    links = member.find_elements(by=By.CSS_SELECTOR, value='a.messageButton')
    
    if len(names) > 0 and len(links) > 0:
        return Member(name=names[0].text, link=links[0].get_attribute('href'))
    else:
        if len(names) > 0:
            print("Names",', '.join([cur_name.text for cur_name in names]), "is skipped")
        return None

In [ ]:
%%time
member_list = [extract_name_and_link(member) for member in membersElems]
len(member_list)

In [ ]:
# all the members who have a link to which we can send a message (for example for myself I cannot)
messagable_members = [member for member in member_list if member is not None]
len(messagable_members)

In [ ]:
#messagable_members[0].name.lower() not in exclude_members
def is_included(cur_name: str):
    cur_name_lower = cur_name.lower()
    return all([ex_mem.lower() not in cur_name_lower for ex_mem in exclude_members])

In [ ]:
members = [member for member in messagable_members if is_included(member.name)]
len(members)

In [ ]:
def extract_first_name(name):
    first_name = name.split(' ')[0].lower()
    first_letter = first_name[0].upper()
    rest_of_name = first_name[1:]
    return first_letter + rest_of_name

In [ ]:
def send_message_to_member(member, sleep_secs = 10, sleep_between_keystrokes=0.05, debug=True):
    driver.get(member.link)
    sleep(sleep_secs)
    textareas = driver.find_elements(by=By.CSS_SELECTOR, value='textarea.composeBox-textArea')
    
    if len(textareas) == 0:
        print(member, "was skipped because for some weird reason no text area was found!!!")
        return
    
    textarea = textareas[0]
    textarea.clear()
    
    
    cur_msg = my_msg.replace('<name>', extract_first_name(member.name))
    if debug:
        print("Current message:",cur_msg)
        
    for letter in cur_msg:
        sleep(sleep_between_keystrokes)
        textarea.send_keys(letter)
    
    buttons = driver.find_elements(by=By.CSS_SELECTOR, value='button.composeBox-sendButton')
    
    if len(buttons) == 0:
        print(member, "was skipped because for some weird reason no button to send message was found!!!")
        return
    
    button = buttons[0]
    button.click()
    
    sleep(sleep_secs)

In [ ]:
driver.maximize_window()

In [ ]:
shifted

In [ ]:
for ii, member in enumerate(members[shifted:]):
    print(ii+shifted)
    try:
        send_message_to_member(member, sleep_between_keystrokes=sleep_between_keystrokes_secs, sleep_secs=sleep_secs_between_messages)
    except ElementNotInteractableException as ex:
        send_message_to_member(member, sleep_between_keystrokes=sleep_between_keystrokes_secs, sleep_secs=sleep_secs_between_messages) # simply retry

In [ ]:
driver.quit()

In [ ]:
# poll(
#     lambda: check(person),
#     step=60*10,
#     poll_forever=True)

# Test